In [1]:
!pip install -Uqqq pip
!pip install -qqq bitsandbytes
!pip install -qqq torch
!pip install -qqq transformers
!pip install -qqq peft
!pip install -qqq datasets
!pip install -qqq loralib
!pip install -qqq einops
!pip install sentence_transformers
!pip install accelerate
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
import json
import os
from pprint import pprint
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
import datasets
from datasets import load_dataset, concatenate_datasets
from transformers import DistilBertTokenizer
from transformers import TFDistilBertForSequenceClassification
from transformers import TextClassificationPipeline
import numpy as np
import tensorflow as tf
import pandas as pd
import json
import gc

from sklearn.model_selection import train_test_split

import re
import evaluate

from huggingface_hub import notebook_login, list_datasets
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer
)

from sentence_transformers import SentenceTransformer, util

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
notebook_login()

# Prep dataset

In [162]:
data_physics = load_dataset("camel-ai/physics")
data_chemistry = load_dataset("camel-ai/chemistry")
data_biology = load_dataset("camel-ai/biology")

In [163]:
# Load dataset

# Convert DatasetDict to pandas DataFrame
data_physics_pandas = data_physics['train'].to_pandas()
data_chemistry_pandas = data_chemistry['train'].to_pandas()
data_biology_pandas = data_biology['train'].to_pandas()


# Fit and transform the column to label encode
data_physics_pandas['subject'] = 'physics'
data_chemistry_pandas['subject'] = 'chemistry'
data_biology_pandas['subject'] = 'biology'

data_pandas = pd.concat([data_physics_pandas, data_chemistry_pandas, data_biology_pandas], ignore_index=True)

data_pandas['labels'] = data_pandas['subject'].astype('category').cat.codes

data_pandas = data_pandas.sample(frac=1).reset_index(drop=True)

data_pandas = data_pandas[['message_1', 'labels']]

In [164]:
import pandas as pd
import numpy as np

# Get unique values in the column
unique_values = data_pandas['labels'].sort_values().unique()

# Create a DataFrame to store the encoded values
encoded_df = pd.DataFrame(columns=unique_values)

# Iterate over each unique value and encode it
for value in unique_values:
    encoded_df[value] = (data_pandas['labels'] == value).astype(float)

data_pandas['labels'] = list(encoded_df.to_numpy())
data_pandas

,message_1,labels
0,How can the unique properties of superconducto...,"[0.0, 0.0, 1.0]"
1,What is the step-by-step reaction mechanism fo...,"[0.0, 1.0, 0.0]"
2,How can the knowledge of chemistry be applied ...,"[0.0, 1.0, 0.0]"
3,How does the ferroelectric behavior of a cryst...,"[0.0, 0.0, 1.0]"
4,What is the Hall coefficient of a silicon semi...,"[0.0, 0.0, 1.0]"
...,...,...
59995,How does soft tissue injury affect the mechani...,"[1.0, 0.0, 0.0]"
59996,"What is the yield of 2,4-dinitrophenol obtaine...","[0.0, 1.0, 0.0]"
59997,A copper rod of length 50 cm and diameter 2 cm...,"[0.0, 0.0, 1.0]"
59998,What is the role of domain walls in the cosmol...,"[0.0, 0.0, 1.0]"


In [165]:
from sklearn.model_selection import train_test_split
from datasets import DatasetDict, Dataset


train_set, test_set = train_test_split(data_pandas, test_size=0.1, random_state=42)
# Assuming `dataframes` is a dictionary containing pandas DataFrames
dataframes_obj = {
    "train": train_set,
    "validation": test_set
}

# Convert pandas DataFrames to DatasetDict
data = DatasetDict()

for key, dataframe in dataframes_obj.items():
    data[key] = Dataset.from_pandas(dataframe)

In [166]:
data

DatasetDict({
    train: Dataset({
        features: ['message_1', 'labels', '__index_level_0__'],
        num_rows: 54000
    })
    validation: Dataset({
        features: ['message_1', 'labels', '__index_level_0__'],
        num_rows: 6000
    })
})

# LOAD MODEL & TOKENIZER

In [167]:
MODEL_NAME = "FacebookAI/roberta-base"

# define label maps
# id2label = {[1,0,0]:"biology", [0,1,0]:"chemistry", [0,0,1]:"physics"}
label2id = {"biology":[1.,0.,0.], "chemistry":[0.,1.,0.], "physics":[0.,0.,1.]}

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=3,
    problem_type="multi_label_classification"
)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [168]:
# create tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, add_prefix_space=True)

# add pad token if none exists
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

In [169]:
# create tokenize function
def tokenize_function(examples):
    # extract text
    text = examples["message_1"]

    #tokenize and truncate text
    tokenizer.truncation_side = "left"
    tokenized_inputs = tokenizer(
        text,
        return_tensors="np",
        truncation=True,
        max_length=512
    )

    return tokenized_inputs

In [170]:
# tokenize training and validation datasets
tokenized_dataset = data.map(tokenize_function, batched=True)
tokenized_dataset

Map:   0%|          | 0/54000 [00:00<?, ? examples/s]

Map:   0%|          | 0/6000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['message_1', 'labels', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 54000
    })
    validation: Dataset({
        features: ['message_1', 'labels', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 6000
    })
})

In [171]:
# create data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# EVALUATE

In [172]:
import numpy as np
from sklearn.metrics import roc_auc_score, f1_score, hamming_loss
from transformers import EvalPrediction
import torch


def multi_labels_metrics(predictions, labels, threshold=0.5):
  sigmoid = torch.nn.Sigmoid()
  probs = sigmoid(torch.Tensor(predictions))

  y_pred = np.zeros(probs.shape)
  y_pred = np.where(probs > threshold, 1, 0).astype(int)
  y_true = labels.astype(int)


  f1 = f1_score(y_true, y_pred, average = 'macro')
  roc_auc = roc_auc_score(y_true, y_pred, average = 'macro')
  hamming = hamming_loss(y_true, y_pred)

  metrics = {
      "roc_auc": roc_auc,
      "hamming_loss": hamming,
      "f1": f1
  }

  return metrics

def compute_metrics(p:EvalPrediction):
  preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions

  result = multi_labels_metrics(predictions=preds,
                                labels=p.label_ids)

  return result

# TRAIN MODEL

In [173]:
peft_config = LoraConfig(task_type="SEQ_CLS",
                        r=4,
                        lora_alpha=32,
                        lora_dropout=0.01,
                        target_modules = ['query'])

In [174]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 666,627 || all params: 125,314,566 || trainable%: 0.531962900466016


In [175]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['message_1', 'labels', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 54000
    })
    validation: Dataset({
        features: ['message_1', 'labels', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 6000
    })
})

# Finetune the model

In [176]:
# hyperparameters
lr = 1e-3
batch_size = 8
num_epochs = 5

# define training arguments
args = TrainingArguments(
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    learning_rate = lr,
    output_dir = './results',
    num_train_epochs=num_epochs,
    save_steps=1000,
    save_total_limit=2
)


# creater trainer object
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# train model
model.config.use_cache = False
trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss
500,0.196600
1000,0.140300
1500,0.152100
2000,0.154500
2500,0.146700
3000,0.156400
3500,0.154800
4000,0.152800
4500,0.166600
5000,0.171000


Checkpoint destination directory ./results/checkpoint-26000 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/checkpoint-27000 already exists and is non-empty. Saving will proceed but saved results may be invalid.


TrainOutput(global_step=33750, training_loss=0.16614628962764033, metrics={'train_runtime': 2620.2614, 'train_samples_per_second': 103.043, 'train_steps_per_second': 12.88, 'total_flos': 1.1093171966034048e+16, 'train_loss': 0.16614628962764033, 'epoch': 5.0})

In [177]:
trainer.evaluate()

{'eval_loss': 0.12277243286371231,
 'eval_roc_auc': 0.957056142194436,
 'eval_hamming_loss': 0.03822222222222222,
 'eval_f1': 0.9428136775769885,
 'eval_runtime': 27.3485,
 'eval_samples_per_second': 219.391,
 'eval_steps_per_second': 27.424,
 'epoch': 5.0}

# Save trained model



In [178]:
model.save_pretrained("trained-model")

In [179]:
PEFT_MODEL = "anishhs001/subject-text-classification-roberta"

model.push_to_hub(
    PEFT_MODEL, use_auth_token=True
)

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:834: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/2.67M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/anishhs001/subject-text-classification-roberta/commit/52fa878f52bcfce97a81dafd4550cb7833f2a41e', commit_message='Upload model', commit_description='', oid='52fa878f52bcfce97a81dafd4550cb7833f2a41e', pr_url=None, pr_revision=None, pr_num=None)

In [181]:
config = PeftConfig.from_pretrained(PEFT_MODEL)
model = AutoModelForSequenceClassification.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    device_map="auto",
    trust_remote_code=True
)

tokenizer=AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

# model = PeftModel.from_pretrained(model, PEFT_MODEL)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Run the finetuned model

In [185]:
%%time
device = "cuda:0"

prompt = """Can you tell me about String theory and its relation to the Schrodinger's equation?"""

print("Trained model predictions:")
print("--------------------------")

# tokenize text
encoding = tokenizer(prompt, return_tensors='pt')
encoding.to(trainer.model.device)

outputs = trainer.model(**encoding)
# compute logits
sigmoid = torch.nn.Sigmoid()
probs = sigmoid(outputs.logits[0].cpu())
preds = np.zeros(probs.shape)
preds[np.where(probs>0.5)] = 1

# Iterate over the items of the dictionary and check if the value satisfies the condition
predictions = [key for key, value in label2id.items() if np.array_equal(value, preds)]

# If there are multiple keys satisfying the condition, all of them will be returned in a list
print("{} : {}".format(prompt,*predictions))

Trained model predictions:
--------------------------
Can you tell me about String theory and its relation to the Schrodinger's equation? : physics
CPU times: user 32.9 ms, sys: 977 µs, total: 33.8 ms
Wall time: 102 ms


# USING THE SPECIALIST MODELS AFTER CATEGORIZATION

In [186]:
def specialist_models(model_name, prompt):

  prompt = """
  <Question>: {}
  <Answer>:
  """.format(prompt).strip()

  bnb_config = BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_use_double_quant=True,
      bnb_4bit_quant_type="nf4",
      bnb_4bit_compute_dtype=torch.bfloat16
  )

  config = PeftConfig.from_pretrained(model_name)

  model = AutoModelForCausalLM.from_pretrained(
      config.base_model_name_or_path,
      return_dict=True,
      quantization_config=bnb_config,
      device_map="auto",
      trust_remote_code=True
  )

  tokenizer=AutoTokenizer.from_pretrained(config.base_model_name_or_path)
  tokenizer.pad_token = tokenizer.eos_token

  model = PeftModel.from_pretrained(model, model_name)

  model.gradient_checkpointing_enable()
  model = prepare_model_for_kbit_training(model)

  generation_config = model.generation_config
  generation_config.max_new_tokens = 1000
  generation_config.temperature = 0.7
  generation_config.top_p = 0.7
  generation_config.num_return_sequences = 1
  generation_config.pad_token_id = tokenizer.eos_token_id
  generation_config.eos_token_id = tokenizer.eos_token_id

  device = "cuda:0"

  encoding = tokenizer(prompt, return_tensors="pt").to(device)
  with torch.inference_mode():
    outputs = model.generate(
        input_ids = encoding.input_ids,
        attention_mask = encoding.attention_mask,
        generation_config = generation_config
    )

  print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [187]:
# if id2label[predictions.tolist()[0]] == "physics":
#   model_name = "anishhs001/physics-falcon-7b"
#   specialist_models(model_name, prompt)
# elif id2label[predictions.tolist()[0]] == "chemistry":
#   model_name = "anishhs001/chemistry-falcon-7b"
#   specialist_models(model_name, prompt)
# else:
#   model_name = "anishhs001/biology-falcon-7b"

model_name = "anishhs001/{}-falcon-7b".format(*predictions)
print(model_name)

anishhs001/physics-falcon-7b


In [189]:
specialist_models(model_name, prompt)

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.
You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.


<Question>: Can you tell me about String theory and its relation to the Schrodinger's equation?
  <Answer>: String theory is a theoretical framework in physics that attempts to unify the principles of general relativity and quantum mechanics. It is based on the idea that the fundamental building blocks of the universe are not particles, but tiny strings that vibrate at different frequencies. These strings are thought to be the smallest possible units of matter and energy, and they are believed to be the fundamental building blocks of the universe.

String theory is related to the Schrodinger's equation because it provides a mathematical framework for describing the behavior of these strings. The Schrodinger's equation is a mathematical equation that describes the behavior of quantum particles, such as electrons and protons, in a quantum mechanical system. In string theory, the Schrodinger's equation is replaced by a modified version of the equation, called the Dirac equation, which des